In [1]:
# Example plotting multiple values
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
import os
import sys
from torchmetrics.image.fid import FrechetInceptionDistance

# sys.path.append("/mnt/raid/C1_ML_Analysis/source/autoencoder/src")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/nets")

from nets import diffusion, spade, lotus, cut, layers, cut_G
from loaders import ultrasound_dataset as usd
import monai

import plotly.express as px
import plotly.graph_objects as go

In [2]:
model = cut.CutG.load_from_checkpoint("/mnt/raid/C1_ML_Analysis/train_output/Cut/allvslast/allvsbutterfly/v0.1/epoch=24-val_loss=5.74.ckpt").eval().cuda()



/mnt/raid/home/jprieto/envs/torch_us/lib/python3.10/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [3]:
img = sitk.ReadImage("Dataset_C1_masked_resampled_256_spc075/2019-01/FAM-025-0001-1_20190121_102442/1.2.840.114340.3.8251017232192.3.20190121.104053.81863.4.nrrd")
img_np = sitk.GetArrayFromImage(img)/255.0

In [4]:
# px.imshow(img_np[:,:,:,0], animation_frame=0, color_continuous_scale='gray', aspect='equal')

In [12]:
with torch.no_grad():
    img_t = torch.tensor(img_np)[:,:,:,0].unsqueeze(1).cuda().to(torch.float32)
    out = []
    for chunk in torch.chunk(img_t, chunks=10):
        out.append(model(chunk).squeeze(1))

In [13]:
out = sitk.GetImageFromArray((torch.cat(out, dim=0).clamp(0, 1).cpu().numpy()*255.0).astype(np.uint8))
sitk.WriteImage(out, "1.2.840.114340.3.8251017232192.3.20190121.104053.81863.4_cutbutterfly.nrrd")
